In [18]:
import pdal
import numpy as np
import json

In [2]:
%load_ext autotime

In [104]:
from shapely.geometry import Point

In [105]:
point = Point([332482.5494008816,7390674.007792895])

In [179]:
# Verificando a resolução mínima
# O tamanho em ângulo do fuso e a distância ao ponto do observador são parâmetros

qt_fases = 60
angulo_fase = 360. / qt_fases
angulo_fase

distancia = np.array([1, 10, 50, 100, 500, 1000, 10000])

np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})
np.sin(np.deg2rad(angulo_fase)) * distancia

array([0.104528, 1.045285, 5.226423, 10.452846, 52.264232, 104.528463,
       1045.284633])

In [197]:
# resolucoes = [[1, 50], [4, 100], [10, 500], [40, 1000], [100, 10000]]
resolucoes = [[2, 550], [4, 1000], [40, 5000], [100, 10000]]

In [206]:
pts = []

In [210]:
for r, d in resolucoes:
    poli = point.buffer(d)
    bounds = poli.bounds
    
    pipeline =  {"pipeline": [{
      "type": "readers.ept",
      "filename": "https://ept-m3dc-pmsp.s3-sa-east-1.amazonaws.com/ept.json",
      "threads": 8,
      "resolution": r,
      "bounds": str(([bounds[0], bounds[2]],[bounds[1], bounds[3]]))
    }]}
    
    r = pdal.Pipeline(json.dumps(pipeline))
    r.validate()
    n_points = r.execute()
    
    if len(pts) > 0:
        pts = np.concatenate([pts, r.arrays[0]], axis=0)
    else:
        pts = r.arrays[0]
    
    print(f'Pipeline selecionou {n_points} pontos em {d}m de distancia ({n_points/poli.area} pts/m2)')

Pipeline selecionou 4563006 pontos em 550m de distancia (4.809209040075934 pts/m2)
Pipeline selecionou 3535766 pontos em 1000m de distancia (1.127279240431746 pts/m2)
Pipeline selecionou 188108 pontos em 5000m de distancia (0.002398917160910898 pts/m2)
Pipeline selecionou 146625 pontos em 10000m de distancia (0.000467472447634564 pts/m2)


In [214]:
pt_edif = pts['Classification'] == 6
pt_ground = pts['Classification'] == 2

In [217]:
len(pts[pt_ground])

153546